<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [1]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-08-25 18:19:50--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-08-25 18:19:50 (16.7 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-08-25 18:19:51--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [5]:
columns=['Name','Market Cap (US$ Billion)']

In [6]:
def extract():
    file_to_process = 'bank_market_cap_1.json'

    extracted_data = extract_from_json(file_to_process)
    extracted_df = pd.DataFrame(extracted_data, columns=columns)

    return extracted_df

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [15]:
# Write your code here
# Load CSV into DataFrame
dataframe = pd.read_csv("exchange_rates.csv", index_col=0)

# Find exchangerate for British pounds
exchange_rate = dataframe.loc["GBP", "Rates"]

print("Exchange Rate for GBP:", exchange_rate)



Exchange Rate for GBP: 0.7323984208000001


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [17]:
def transform(dataframe, exchange_rate):
    # Convert Market Cap from USD to GBP
    dataframe["Market Cap (GBP$ Billion)"] = dataframe["Market Cap (US$ Billion)"] * exchange_rate
    # Round the Market Cap to 3 decimal places
    dataframe["Market Cap (GBP$ Billion)"] = dataframe["Market Cap (GBP$ Billion)"].round(3)
    # Drop the original Market Cap column
    dataframe.drop(columns=["Market Cap (US$ Billion)"], inplace=True)

    return dataframe


# Given exchange rate for GBP
exchange_rate = 0.7323984208000001

# Extracted dataframe
extracted_df = extract()

# Apply transformation
transformed_df = transform(extracted_df, exchange_rate)

# Print the transformed dataframe
print(transformed_df)

                                       Name  Market Cap (GBP$ Billion)
0                            JPMorgan Chase                    286.319
1   Industrial and Commercial Bank of China                    252.834
2                           Bank of America                    238.272
3                               Wells Fargo                    225.588
4                   China Construction Bank                    188.519
..                                      ...                        ...
65                             Ping An Bank                     27.826
66                       Standard Chartered                     27.332
67                     United Overseas Bank                     25.728
68                                QNB Group                     24.579
69                              Bank Rakyat                     24.228

[70 rows x 2 columns]


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [18]:
def load(dataframe, filename):
    dataframe.to_csv(filename, index=False)

# using the transformed_df
load(transformed_df, "bank_market_cap_gbp.csv")

## Logging Function


Write the logging function <code>log</code> to log your data:


In [20]:
def log(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"[{timestamp}] {message}\n"
    
    with open("log.txt", "a") as log_file:
        log_file.write(log_message)


## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [21]:
def extract():
    log("ETL Job Started")

    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Completed")

    return extracted_data


# Run the extract process
extracted_data = extract()


### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [22]:
# Call function
extracted_data = extract()

# Print the first 5 rows
print(extracted_data.head())

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399


Log the data as <code>"Extract phase Ended"</code>


In [23]:
def extract():
    log("ETL Job Started")
    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Ended")

    return extracted_data


### Transform


Log the following  <code>"Transform phase Started"</code>


In [25]:
def extract_and_transform():
    log("ETL Job Started")

    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Ended")

    exchange_rate = 0.7323984208000001

    log("Transform phase Started")
    extracted_df = pd.DataFrame(extracted_data, columns=['Name', 'Market Cap (US$ Billion)'])
    transformed_df = transform(extracted_df, exchange_rate)

    return transformed_df


<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [26]:
transformed_data = extract_and_transform()

print(transformed_data.head())

                                      Name  Market Cap (GBP$ Billion)
0                           JPMorgan Chase                    286.319
1  Industrial and Commercial Bank of China                    252.834
2                          Bank of America                    238.272
3                              Wells Fargo                    225.588
4                  China Construction Bank                    188.519


Log your data <code>"Transform phase Ended"</code>


In [27]:
def extract_and_transform():
    log("ETL Job Started")

    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Ended")

    exchange_rate = 0.7323984208000001

    log("Transform phase Started")
    extracted_df = pd.DataFrame(extracted_data, columns=['Name', 'Market Cap (US$ Billion)'])
    transformed_df = transform(extracted_df, exchange_rate)
    log("Transform phase Ended")

    return transformed_df


### Load


Log the following `"Load phase Started"`.


In [28]:
def extract_and_transform():
    log("ETL Job Started")

    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Ended")

    exchange_rate = 0.7323984208000001

    log("Transform phase Started")
    extracted_df = pd.DataFrame(extracted_data, columns=['Name', 'Market Cap (US$ Billion)'])
    transformed_df = transform(extracted_df, exchange_rate)
    log("Transform phase Ended")

    log("Load phase Started")
    load(transformed_df, "bank_market_cap_gbp.csv")

    return transformed_df


Call the load function


In [29]:
def extract_and_transform():
    log("ETL Job Started")

    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Ended")

    exchange_rate = 0.7323984208000001

    log("Transform phase Started")
    extracted_df = pd.DataFrame(extracted_data, columns=['Name', 'Market Cap (US$ Billion)'])
    transformed_df = transform(extracted_df, exchange_rate)
    log("Transform phase Ended")

    log("Load phase Started")
    load(transformed_df, "bank_market_cap_gbp.csv")

    return transformed_df


Log the following `"Load phase Ended"`.


In [30]:
def extract_and_transform():
    log("ETL Job Started")

    file_to_process = 'bank_market_cap_1.json'
    log("Extract phase Started")
    extracted_data = extract_from_json(file_to_process)
    log("Extract phase Ended")

    exchange_rate = 0.7323984208000001

    log("Transform phase Started")
    extracted_df = pd.DataFrame(extracted_data, columns=['Name', 'Market Cap (US$ Billion)'])
    transformed_df = transform(extracted_df, exchange_rate)
    log("Transform phase Ended")

    log("Load phase Started")
    load(transformed_df, "bank_market_cap_gbp.csv")
    log("Load phase Ended")

    return transformed_df


## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
